In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from time import time

In [4]:
filename = r'Global_Mobility_Report.zip'
mobility_wide = pd.read_csv(
    filename,
    compression='zip'   
)

C:\Users\USANCOL3\AppData\Local\Continuum\anaconda3\envs\analysis\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
country_region_code = 'US'
mobility_wide = mobility_wide.loc[
    mobility_wide.country_region_code == country_region_code
]

In [ ]:
mobility_wide.columns

In [ ]:
mobility_wide = mobility_wide.rename(columns={
    'retail_and_recreation_percent_change_from_baseline': 'retail_recreation',
    'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_pharmacy',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline': 'transit_stations',
    'workplaces_percent_change_from_baseline': 'workplaces',
    'residential_percent_change_from_baseline': 'residential'
})

In [ ]:
mobility_wide.dtypes

In [ ]:
mobility_wide.memory_usage(deep=True) / 1024**2

In [ ]:
mobility_wide.isna().sum()

In [ ]:
mobility_wide.head()

In [ ]:
def memory_usage(df, detail=False):
    mem = df.memory_usage(deep=True) / 1024**2
    tot_mem = round(mem.sum(), 2)
    print(f"total memory MB: {tot_mem}")
    if detail:
        print(f"\n{mem}")

In [ ]:
memory_usage(mobility_wide, detail=True)

In [ ]:
mobility_wide.date = pd.to_datetime(mobility_wide.date)

In [ ]:
mobility_long = mobility_wide.melt(id_vars=[
    'country_region_code',
    'sub_region_1',
    'sub_region_2',
    'metro_area',
    'census_fips_code',
    'date'
    ], 
    value_vars=[
    'retail_recreation',
    'grocery_pharmacy',
    'parks',
    'transit_stations',
    'workplaces',
    'residential'
    ],
    var_name='activity',
    value_name='percent_change'
)

In [ ]:
memory_usage(mobility_long, detail=True)

In [ ]:
# write to file
start = time()
mobility_long.to_csv(
    f'{country_code}_mobility_long.csv.gz',
    compression='gzip',
    index=False
)
end = time()
print(end - start)

In [ ]:
start = time()
us_long = pd.read_csv('./US_mobility_long.zip', compression='zip', parse_dates=['date'])
end = time()
print(end - start)
us_long.head()

In [ ]:
us_long.dtypes

In [ ]:
us = mobility_long.loc[
    (mobility_long.country_region == 'United States') &
    (mobility_long.census_fips_code.notna())
]

In [ ]:
us.head()

In [ ]:
us.loc[
    (us.sub_region_1 == 'Alabama') &
    (us.sub_region_2.isin(['Autauga County', 'Baldwin County'])),
    ['date']
].value_counts()

In [ ]:
co.head()

In [ ]:
co.shape

In [ ]:
boulder = co.loc[
    co.sub_region_2 == 'Boulder County'
]

In [ ]:
boulder.columns

In [ ]:
boulder.activity.unique()

In [ ]:
boulder_transit = boulder.loc[
    boulder.activity == 'retail_recreation',
    [
        'date',
        'percent_change'
    ]
]

In [ ]:
# function for shading weekend days in plots
def highlight_weekends(ax, timeseries):
    # access datetime attributes of timeseries
    dt = timeseries.dt
    # create df of weekend start and end dates
    ranges = timeseries[dt.dayofweek >= 5].groupby([dt.year, dt.isocalendar().week]).agg(['min', 'max'])
    # fill area between weekend dates
    for i, tmin, tmax in ranges.itertuples():
        ax.axvspan(tmin, tmax, facecolor='gray', edgecolor='none', alpha=0.1)

In [ ]:
fig, ax = plt.subplots(figsize=(23, 6))
sns.lineplot(x=co.date.unique(), y=boulder_transit.percent_change)
highlight_weekends(ax, co.date)
# ax = sns.lineplot(x=co.date.unique(), y=boulder_transit.percent_change.rolling(7).mean())
# start, end = ax.get_xlim()
# ax.xaxis.set_ticks(np.arange(start + 9, end - 4, 7))
# plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
ax.xaxis.set_tick_params(length=0)
ax.yaxis.set_tick_params(length=0)
plt.xlabel("year-month")
plt.title("transit")
sns.despine(left=True, bottom=True)
plt.show()